# 爬取指定类型电视剧

In [46]:
#抓取数据进行解析
# https://m.douban.com/rexxar/api/v2/tv/recommend?refresh=0&start=0&count=20&selected_categories=%7B%22%E5%9C%B0%E5%8C%BA%22:%22%E5%8D%8E%E8%AF%AD%22%7D&uncollect=false&tags=%E5%8D%8E%E8%AF%AD
# https://m.douban.com/rexxar/api/v2/tv/recommend?refresh=0&start=20&count=20&selected_categories=%7B%22%E5%9C%B0%E5%8C%BA%22:%22%E5%8D%8E%E8%AF%AD%22%7D&uncollect=false&tags=%E5%8D%8E%E8%AF%AD
import urllib

url_init1 = ('https://m.douban.com/rexxar/api/v2/tv/recommend?refresh=0&start=0&count=20&selected_categories=%7B%22%E5%9C%B0%E5%8C%BA%22:%22%E5%8D%8E%E8%AF%AD%22%7D&uncollect=false&tags=%E5%8D%8E%E8%AF%AD')
url_init2=('https://m.douban.com/rexxar/api/v2/tv/recommend?refresh=0&start=20&count=20&selected_categories=%7B%22%E5%9C%B0%E5%8C%BA%22:%22%E5%8D%8E%E8%AF%AD%22%7D&uncollect=false&tags=%E5%8D%8E%E8%AF%AD')
url1 = urllib.parse.unquote(url_init1)
url2=urllib.parse.unquote(url_init2)
print(url1)
print(url2)

https://m.douban.com/rexxar/api/v2/tv/recommend?refresh=0&start=0&count=20&selected_categories={"地区":"华语"}&uncollect=false&tags=华语
https://m.douban.com/rexxar/api/v2/tv/recommend?refresh=0&start=20&count=20&selected_categories={"地区":"华语"}&uncollect=false&tags=华语


In [47]:
#获取对应的参数
query = urllib.parse.urlparse(url2).query
params = urllib.parse.parse_qs(query)
params

{'refresh': ['0'],
 'start': ['20'],
 'count': ['20'],
 'selected_categories': ['{"地区":"华语"}'],
 'uncollect': ['false'],
 'tags': ['华语']}

In [20]:
import requests
import json
import time
import pandas as pd


def get_one_request(url, headers, params):
    res = None
    cnt = 0
    while cnt <= 3:
        req = requests.get(url, headers=headers, params=params)
        if req.status_code == 200:
            res = json.loads(req.text)
            #res保存为json文件
            with open('douban.json', 'w', encoding='utf-8') as f:
                json.dump(res, f, ensure_ascii=False, indent=4)
            break
        else:
            cnt += 1
            continue
          
    return res

def get_origin_data(page_cnt):
    res_list = []
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.41',
        'Referer': 'https://movie.douban.com/explore'
    }
    url_base = 'https://m.douban.com/rexxar/api/v2/tv/recommend?'
    params = {
        'refresh': ['0'],
        'start': ['0'],
        'count': ['20'],
        'selected_categories': ['{"地区":"华语"}'],
        'uncollect': ['false'],
        'tags': ['华语']
    }
    for i in range(page_cnt):
        start = str(i * 20)
        params['start'] = [start]
        res = get_one_request(url_base, headers, params)
        res_list.append(res)
        time.sleep(1)
      
    return res_list


if __name__ == '__main__':
    page_cnt = 1
    data = get_origin_data(page_cnt)
    data_clean = [j for i in range(len(data)) 
                  for j in data[i]['items']]
    df = pd.DataFrame(data_clean)
    df[['title', 'year', 'card_subtitle']]

In [7]:
# df是json格式的数据，需要将其转换为csv格式
df.to_csv('douban.csv', index=False, encoding='utf-8')

In [17]:
# df保存为json格式
df.to_json('douban.json', orient='records', force_ascii=False)

In [23]:
f=open('douban.json','r',encoding='utf-8')
data=json.load(f)

In [27]:
for i in data['items']:
    print(i['title'],i['year'],i['card_subtitle'],i['rating']['star_count'])

花儿与少年·丝路季 2023 2023 / 中国大陆 / 真人秀 / 李超 / 秦海璐 秦岚 4.5
繁花 2023 2023 / 中国大陆 / 剧情 爱情 / 王家卫 / 胡歌 马伊琍 4.5
要久久爱 2024 2024 / 中国大陆 / 剧情 / 牛超 马成成 / 杨紫 范丞丞 0.0
仙剑四 2024 2024 / 中国大陆 / 剧情 奇幻 古装 / 杨玄 黄埜 / 鞠婧祎 陈哲远 2.0
大江大河之岁月如歌 2024 2024 / 中国大陆 / 剧情 / 孔笙 孙墨龙 刘洪源 / 王凯 杨烁 4.0
祈今朝 2024 2024 / 中国大陆 / 剧情 爱情 奇幻 古装 / 刘国楠 / 许凯 虞书欣 2.5
你也有今天 2024 2024 / 中国大陆 / 剧情 爱情 / 陈铭章 陈世峄 / 陈星旭 章若楠 3.5
如果奔跑是我的人生 2024 2024 / 中国大陆 / 剧情 / 沈严 李江明 / 钟楚曦 杨超越 3.5
漫长的季节 2023 2023 / 中国大陆 / 剧情 家庭 犯罪 / 辛爽 / 范伟 秦昊 4.5
莲花楼 2023 2023 / 中国大陆 / 剧情 悬疑 武侠 古装 / 郭虎 任海涛 / 成毅 曾舜晞 4.5
在暴雪时分 2024 2024 / 中国大陆 / 剧情 爱情 / 黄天仁 / 吴磊 赵今麦 0.0
黑土无言 2024 2024 / 中国大陆 / 剧情 悬疑 犯罪 / 臧溪川 / 陈建斌 胡军 3.5
三大队 2023 2023 / 中国大陆 / 动作 悬疑 犯罪 / 邢键钧 / 秦昊 李乃文 3.5
狂飙 2023 2023 / 中国大陆 / 剧情 犯罪 / 徐纪周 / 张译 张颂文 4.5
长相思 2023 2023 / 中国大陆 / 爱情 奇幻 古装 / 秦榛 杨欢 / 杨紫 张晚意 4.0
狗剩快跑 2024 2024 / 中国大陆 / 喜剧 历史 战争 / 王新军 / 蒋龙 史策 0.0
后宫·甄嬛传 2011 2011 / 中国大陆 / 剧情 古装 / 郑晓龙 / 孙俪 陈建斌 4.5
神隐 2023 2023 / 中国大陆 / 剧情 爱情 奇幻 古装 / 陈家霖 李才 / 赵露思 王安宇 3.0
长歌行 2021 2021 / 中国大陆 / 剧情 古装 / 朱锐斌 / 迪丽热巴 吴磊 3.0
新闻女王 2023

In [ ]:
#访问网站获取数据
import csv
import requests
import json
import os

def Get_request(url,headers,params):
    print('tt')
    try:
        print('hh')
        r=requests.get(url,headers=headers,params=params,timeout=30)
        r.raise_for_status()
        print(r.status_code)        
        res = json.loads(r.text)
        return res
    except:
        print('error')
        return 'ERROR!!'

def Save_file(res):
    if not os.path.exists('images'):
        os.makedirs('images')

    with open('DoubanTv.csv', 'a', newline='', encoding='utf-8') as dbtv:
        writer = csv.writer(dbtv)
        for item in res['items']:
            Info = (
                item['id'],
                item['title'],
                item['rating']['value'],
                item['card_subtitle'],
                item['year'],
                item['pic']['large']
            )
            writer.writerow(Info)

            # Download and save the image
            image_url = item['pic']['large']
            response = requests.get(image_url)
            if response.status_code == 200:
                with open(f'images/{item["id"]}.jpg', 'wb') as img:
                    img.write(response.content)
        

def Get_data(page_cnt):
    base_url='https://m.douban.com/rexxar/api/v2/tv/recommend?'
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36(KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36',
               'Referer': 'https://movie.douban.com/explore'}
    params = {
        'refresh': ['0'],
        'start': ['0'],
        'count': ['20'],
        'selected_categories': ['{"地区":"华语"}'],
        'uncollect': ['false'],
        'tags': ['华语']
    }
    try:
        for i in range(page_cnt):
            start = str(i * 20)
            params['start'] = [start]
            print(base_url,params)
            res=Get_request(base_url,headers,params)
            Save_file(res)
            time.sleep(5)
    except:
        return 'Get error maybe have done!!'

          
def main():
    page_cnt=100
    Get_data(page_cnt)

if __name__=='__main__':
    main()

In [97]:
#读入DoubanTv.csv
import pandas as pd

tv_data=pd.read_csv(r'D:\000zyf\Learning\python_learn\spider\script\练习\douban_tv_filter\DoubanTv.csv')